Imports

In [1]:
import os
os.environ["PYTHONWARNINGS"] = "ignore"


In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from scipy.stats import loguniform

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score


Config

In [3]:
PROJECT_ROOT = Path("/Users/srinivass/Budgetaware_hpo")

HPO_DIR = PROJECT_ROOT / "results" / "hpo"
HPO_DIR.mkdir(parents=True, exist_ok=True)

DATASET_NAME = "covertype"

TEST_SIZE = 0.2
VAL_SIZE = 0.25   # 0.25 of 80% → 20% validation

N_RUNS = 10
RUN_SEEDS = list(range(N_RUNS))

N_ITER_RANDOM = 20   # random configs per run

print("Saving HPO results to:", HPO_DIR)


Saving HPO results to: /Users/srinivass/Budgetaware_hpo/results/hpo


Dataset Loader

In [4]:
def load_covertype():
    X, y = fetch_openml(
        name="covertype",
        version=2,
        as_frame=False,
        return_X_y=True
    )
    return X, y.astype(int)

X, y = load_covertype()

from sklearn.utils import resample

MAX_SAMPLES = 50000  # adjust later if needed

if X.shape[0] > MAX_SAMPLES:
    X, y = resample(
        X, y,
        n_samples=MAX_SAMPLES,
        stratify=y,
        random_state=42
    )

print("Using dataset shape:", X.shape)



Using dataset shape: (50000, 54)


MLP Builder

In [5]:
def build_mlp(params, max_iter, random_state):
    return Pipeline([
        ("scaler", StandardScaler(with_mean=False)),
        ("mlp", MLPClassifier(
            hidden_layer_sizes=params["hidden_layer_sizes"],
            activation=params["activation"],
            learning_rate_init=params["learning_rate_init"],
            alpha=params["alpha"],
            batch_size=params["batch_size"],
            solver="adam",
            max_iter=max_iter,
            early_stopping=False,
            random_state=random_state
        ))
    ])


Hyperparameter Search Spaces

In [6]:
PARAM_SPACE = {
    "hidden_layer_sizes": [
        (50,), (100,), (200,),
        (100, 100), (200, 100)
    ],
    "activation": ["relu", "tanh"],
    "learning_rate_init": loguniform(1e-4, 5e-2),
    "alpha": loguniform(1e-6, 1e-1),
    "batch_size": [64, 128, 256, 512],
}


In [7]:
def sample_params(rng):
    return {
        "hidden_layer_sizes": PARAM_SPACE["hidden_layer_sizes"][
            rng.randint(len(PARAM_SPACE["hidden_layer_sizes"]))
        ],
        "activation": PARAM_SPACE["activation"][
            rng.randint(len(PARAM_SPACE["activation"]))
        ],
        "learning_rate_init": PARAM_SPACE["learning_rate_init"].rvs(random_state=rng),
        "alpha": PARAM_SPACE["alpha"].rvs(random_state=rng),
        "batch_size": PARAM_SPACE["batch_size"][
            rng.randint(len(PARAM_SPACE["batch_size"]))
        ],
    }


Random Search Function

In [9]:
def run_random_search(X_train, y_train, X_val, y_val, seed):
    rng = np.random.RandomState(seed)

    best_score = -np.inf
    best_params = None

    for i in range(N_ITER_RANDOM):
        print(f"  RS config {i+1}/{N_ITER_RANDOM}", end="\r")

        params = sample_params(rng)
        model = build_mlp(params, max_iter=300, random_state=seed)

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        score = f1_score(y_val, preds, average="macro")

        if score > best_score:
            best_score = score
            best_params = params

    print()  # newline after progress
    return best_score, best_params


Successive Halving Function

In [10]:
def run_successive_halving(X_train, y_train, X_val, y_val, seed):
    rng = np.random.RandomState(seed)

    candidates = [sample_params(rng) for _ in range(12)]
    budgets = [30, 100, 300]

    for budget in budgets:
        print(f"  SHA budget={budget}, candidates={len(candidates)}")

        scores = []
        for params in candidates:
            model = build_mlp(params, max_iter=budget, random_state=seed)
            model.fit(X_train, y_train)

            preds = model.predict(X_val)
            score = f1_score(y_val, preds, average="macro")
            scores.append(score)

        # keep top 1/3
        k = max(1, len(candidates) // 3)
        top_idx = np.argsort(scores)[-k:]
        candidates = [candidates[i] for i in top_idx]

    # final evaluation among remaining candidates
    best_score = max(scores)
    best_params = candidates[np.argmax(scores)]
    return best_score, best_params


Main Loop over Datasets and Methods

In [11]:
results = []

for seed in RUN_SEEDS:
    print(f"\n=== HPO Run {seed + 1}/{N_RUNS} ===")

    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=seed
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=VAL_SIZE, stratify=y_temp, random_state=seed
    )

    rs_score, rs_params = run_random_search(X_train, y_train, X_val, y_val, seed)
    sh_score, sh_params = run_successive_halving(X_train, y_train, X_val, y_val, seed)

    results.append({
        "dataset": DATASET_NAME,
        "seed": seed,
        "method": "random_search",
        "val_f1": rs_score
    })

    results.append({
        "dataset": DATASET_NAME,
        "seed": seed,
        "method": "successive_halving",
        "val_f1": sh_score
    })

    print(f"RS F1={rs_score:.4f} | SH F1={sh_score:.4f}")



=== HPO Run 1/10 ===


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


...


  SHA budget=30, candidates=12


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged ye

  SHA budget=100, candidates=4


/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/srinivass/Budgetaware_hpo/.venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


  SHA budget=300, candidates=1
RS F1=0.8706 | SH F1=0.8682


Save Results to CSV

In [12]:
df_hpo = pd.DataFrame(results)

output_path = HPO_DIR / "mlp_hpo_summary_covertype.csv"
df_hpo.to_csv(output_path, index=False)

print("\nSaved HPO summary to:")
print(output_path)

df_hpo.groupby("method")["val_f1"].describe()



Saved HPO summary to:
/Users/srinivass/Budgetaware_hpo/results/hpo/mlp_hpo_summary_covertype.csv


,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
random_search,10.0,0.878019,0.005944,0.870550,0.872627,0.878309,0.881626,0.887256
successive_halving,10.0,0.866383,0.016206,0.828663,0.866977,0.868634,0.876235,0.885487
